In [ ]:

%run includes/3-network-setup.ipynb

# Networks (Networks and Subnetworks)
Summary and detail reports on networks and subnetworks deployed across the GCP estate.

## Networks

In [ ]:

# get and show networks
network_queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    JSON_ARRAY_LENGTH(peerings) as num_peerings,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    JSON_ARRAY_LENGTH(subnetworks) as num_subnets
    FROM google.compute.networks WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in all_projects
]
networks_df = run_stackql_queries(network_queries)
networked_projects = list(networks_df['project'].unique())
networks_df

## Subnetworks

In [ ]:

# get and show subnetworks
queries = [
    f"""
    SELECT 
    name,
    SPLIT_PART(region, '/', 9) as region,
    SPLIT_PART(network, '/', 10) as network,
    '{project}' as project,
    stackType,
    ipCidrRange,
    secondaryIpRanges,
    gatewayAddress,
    ipv6CidrRange,
    purpose,
    enableFlowLogs,
    privateIpGoogleAccess
    FROM google.compute.subnetworks WHERE project = '{project}'
    """
    for project in networked_projects
]
subnetworks_df = run_stackql_queries(queries)
subnetworks_df

### Subnet IAM Policies

In [ ]:

# get subnet iam policies
queries = [
    f"""
    SELECT 
    '{row['name']}' as subnet
    ,'{row['project']}' as project
    ,'{row['region']}'  as region
    , role
    , condition
    , SPLIT_PART(members.value, ':', 1) as member_type
    , SPLIT_PART(members.value, ':', 2) as member
    FROM google.compute.subnetworks_iam_policies, json_each(members) members
    WHERE project = '{row['project']}' AND region = '{row['region']}' AND resource = '{row['name']}'
    """
    for _, row in subnetworks_df.iterrows()
]
subnetworks_bindings_df = run_stackql_queries(queries)
subnetworks_bindings_df

## Network Peerings

In [ ]:

# get network peerings
network_peering_queries = [
    f"""
    SELECT 
    JSON_EXTRACT(peerings.value, '$.name') as peering_name,
    name,
    SPLIT_PART(JSON_EXTRACT(peerings.value, '$.network'), '/', 10) as peering_network,
    '{project}' as project,
    IIF((JSON_EXTRACT(peerings.value, '$.autoCreateRoutes') = 1), 'true', 'false') as autoCreateRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.exchangeSubnetRoutes') = 1), 'true', 'false') as exchangeSubnetRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.importCustomRoutes') = 1), 'true', 'false') as importCustomRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.exportCustomRoutes') = 1), 'true', 'false') as exportCustomRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.importSubnetRoutesWithPublicIp') = 1), 'true', 'false') as importSubnetRoutesWithPublicIp,
    IIF((JSON_EXTRACT(peerings.value, '$.exportSubnetRoutesWithPublicIp') = 1), 'true', 'false') as exportSubnetRoutesWithPublicIp,
    JSON_EXTRACT(peerings.value, '$.stackType') as stackType,
    JSON_EXTRACT(peerings.value, '$.state') as state,
    JSON_EXTRACT(peerings.value, '$.stateDetails') as stateDetails
    FROM google.compute.networks, json_each(peerings) peerings
    WHERE project = '{project}'
    AND peerings IS NOT NULL
    """
    for project in networked_projects
]
network_peerings_df = run_stackql_queries(network_peering_queries)
network_peerings_df

### Network Peering Graph

In [ ]:

# graph peerings
graph_network_peerings(network_peerings_df)

## Shared VPC

In [ ]:

# get xpn networks
queries = [
    f"""
   SELECT 
   name 
   FROM 
   google.compute.projects
   WHERE project =  '{project}' AND xpnProjectStatus = 'HOST'
    """
    for project in networked_projects
]
xpn_hosts_df = run_stackql_queries(queries)
xpn_hosts = xpn_hosts_df['name'].tolist()

queries = [
    f"""
   SELECT 
   id as serviceProject
   ,'{project}' as hostProject
   FROM 
   google.compute.xpn_resources
   WHERE project =  '{project}'
    """
    for project in xpn_hosts
]
xpn_resources_df = run_stackql_queries(queries)

In [ ]:

# show tree
tree = generate_xpn_tree(xpn_resources_df)
tree

## Addresses

In [ ]:

# get addresses
address_queries = [
    f"""
    SELECT 
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 10) as network,
    SPLIT_PART(subnetwork, '/', 11) as subnetwork,
    addressType,
    address,
    ipVersion,
    ipv6EndpointType,
    purpose,
    labels,
    status,
    SPLIT_PART(JSON_EXTRACT(users, '$[0]'), '/', 11) as user,
    networkTier
    FROM google.compute.addresses WHERE project = '{project}'
    """
    for project in networked_projects
]
addresses_df = run_stackql_queries(address_queries)

global_address_queries = [
    f"""
    select 
    name,
    description,
    '{project}' as project,
    SPLIT_PART(network, '/', 10) as network,
    labels,
    networkTier,
    users,
    addressType,
    address,
    purpose,
    ipVersion,
    prefixLength,
    status,
    subnetwork,
    ipv6EndpointType
    from google.compute.global_addresses where project = '{project}' and name is not null
    """
    for project in networked_projects
]
global_address_df = run_stackql_queries(global_address_queries)

### Address Details

In [ ]:

# show addresses
addresses_df

### Global Address Details

In [ ]:

# show global addresses
global_address_df